In [120]:
import joblib
import numpy as np
import pandas as pd
# from ipywidgets import interact, widgets
from IPython.display import display
from ipywidgets import widgets, Output, VBox, Button, HBox, Textarea

In [121]:
model = joblib.load('bank_loan.joblib', mmap_mode=None)

In [122]:
data = joblib.load('df.joblib')

In [123]:
mapping = joblib.load('encoded_mapping.joblib')

In [124]:
data.head()


,Age,Gender,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Home Ownership,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,M,1,49.0,91107,4,1.6,1,0,Home Owner,0,1,0,0,0
1,45,M,19,34.0,90089,3,1.5,1,0,Rent,0,1,0,0,0
2,39,M,15,11.0,94720,1,1.0,1,0,Rent,0,0,0,0,0
3,35,M,9,100.0,94112,1,2.7,2,0,Rent,0,0,0,0,0
4,35,M,8,45.0,91330,4,1.0,2,0,Rent,0,0,0,0,1


In [125]:
print("Mappings can be seen as")
print()
def print_mapping(mapping):
    for key, values in mapping.items():
        print(f"{key}:")
        for index, value in enumerate(values):
            if index %12 ==0:
              print()

            print(f"  {index}. {value}", end='')
        print() # Add an empty line between different categories
print_mapping(mapping)

Mappings can be seen as

Gender:

  0. F  1. M  2. O
ZIP Code:

  0. 90005  1. 90007  2. 90009  3. 90011  4. 90016  5. 90018  6. 90019  7. 90024  8. 90025  9. 90027  10. 90028  11. 90029
  12. 90032  13. 90033  14. 90034  15. 90035  16. 90036  17. 90037  18. 90041  19. 90044  20. 90045  21. 90048  22. 90049  23. 90057
  24. 90058  25. 90059  26. 90064  27. 90065  28. 90066  29. 90068  30. 90071  31. 90073  32. 90086  33. 90089  34. 90095  35. 90210
  36. 90212  37. 90230  38. 90232  39. 90245  40. 90250  41. 90254  42. 90266  43. 90272  44. 90274  45. 90275  46. 90277  47. 90280
  48. 90291  49. 90304  50. 90401  51. 90404  52. 90405  53. 90502  54. 90503  55. 90504  56. 90505  57. 90509  58. 90601  59. 90623
  60. 90630  61. 90638  62. 90639  63. 90640  64. 90650  65. 90717  66. 90720  67. 90740  68. 90745  69. 90747  70. 90755  71. 90813
  72. 90840  73. 91006  74. 91007  75. 91016  76. 91024  77. 91030  78. 91040  79. 91101  80. 91103  81. 91105  82. 91107  83. 91109
  84. 91116  85

In [126]:

input_widgets = []




In [142]:
categorical = {
    'Education': {'Undergrad':1, 'Graduate':2, 'Advanced/Professional':3},
    'Securities Account': {'Yes':1, 'No':0},
    'CD Account': {'Yes':1, 'No':0},
    'Online': {'Yes':1, 'No':0},
    'CreditCard': {'Yes':1, 'No':0},
    'Family' :{1:1,2:2,3:3,4:4},
    'Gender' :{"Male":1,"Female":0,'Others':2},
    'Mortgage' :{"Yes":1,"No":0},
    'Home Ownership' :{'Home Owner':1,'Rent':2,'Home Mortgage':0},
}


input_text = widgets.Text(
    value='',
    placeholder='Type something',
    description='You:',
    disabled=False
)

output_text = Output()
user_inputs = {}
input_widgets = []

def get_user_input_widgets(data, categorical = categorical):
    global input_widgets
    input_widgets = []
    cols = data.drop('Personal Loan', axis=1).columns
    for col in cols:
        if col in categorical:
            if isinstance(categorical[col], dict):
                # If the column has specific mappings, use Dropdown widget
                input_widgets.append(
                    widgets.Dropdown(
                        options=categorical[col],
                        description=f'{col}:',
                        disabled=False
                    )
                )
            else:
                # If the column has a list of values, use Dropdown widget
                input_widgets.append(
                    widgets.Dropdown(
                        options=categorical[col],
                        description=f'{col}:',
                        disabled=False
                    )
                )
        else:
            # For non-categorical columns, use Text widget
            input_widgets.append(
                widgets.Text(
                    value='',
                    placeholder=f'Enter data for {col}',
                    description=f'{col}:',
                    disabled=False
                )
            )

def predict(data, model):
    user_input = {w.description[:-1]: w.value for w in input_widgets}
    user_input_df = pd.DataFrame([user_input])  # Create DataFrame from collected inputs
    # Ensure the columns match those expected by the model
    user_input_df.columns = [col.strip() for col in user_input_df.columns]
    user_input_df = user_input_df.reindex(columns=data.drop('Personal Loan', axis=1).columns)
    prediction = model.predict(user_input_df)
    return "Congrats! Personal Loan can be credited to You!" if prediction[0] else "Sorry, Personal Loan cannot be credited to You."


In [144]:
# Define chatbot function
def chatbot(input_text, data, model):
  try:
      user_input = input_text
      categorical = data.select_dtypes(include=['object']).columns
      if 'hello' in user_input:
          return "Hi there! How can I help you today?"

      elif 'predict' in user_input:
          get_user_input_widgets(data)
          submit_button = Button(description="Submit")
          submit_button.on_click(on_submit)
          display(VBox(input_widgets + [submit_button]))
          return f"Please enter the required data.\n Also please, give the encoded data for {categorical} and Education column "

      elif 'mean' in user_input:
          column = user_input.split('mean of ')[-1]
          if column in data.columns:
              mean_value = data[column].mean()
              return f"The mean of {column} is {mean_value:.2f}"
          else:
              return f"Column {column} not found in the dataset. Also try capitalizing the Column's name"

      elif 'median' in user_input:
          column = user_input.split('median of ')[-1]
          if column in data.columns:
              median_value = data[column].median()
              return f"The median of {column} is {median_value:.2f}"
          else:
              return f"Column {column} not found in the dataset. Also try capitalizing the Column's name"

      elif 'mode' in user_input:
          column = user_input.split('mode of ')[-1]
          if column in data.columns:
              mode_value = data[column].mode()[0]
              return f"The mode of {column} is {mode_value}"
          else:
              return f"Column {column} not found in the dataset. Also try capitalizing the Column's name"
      elif 'describe' in user_input:
          return data.describe().to_string()
      else:
          return "I'm sorry, I didn't understand that. Can you please clarify?"
  except Exception as e:
      print(e)

# Function to handle submit button click
def on_submit(b):
    with output_text:
        output_text.clear_output()
        prediction = predict(data, model)
        print(f'Chatbot: {prediction}')

# Function to handle text input submission
def on_text_submit(change):
    with output_text:
        output_text.clear_output()
        response = chatbot(change['new'], data, model)  # Pass user input, data, and model
        print(f'Chatbot: {response}')

# Create interactive widgets
input_text = widgets.Text(
    value='',
    placeholder='Type something',
    description='You:',
    disabled=False
)

output_text = Output()

input_text.observe(on_text_submit, names='value')

# Display widget
display(VBox([input_text, output_text]))